In [1]:
pip install pymongo pandas openpyxl

  Using cached dnspython-2.7.0-py3-none-any.whl.metadata (5.8 kB)
   ---------------------------------------- 0.0/797.9 kB ? eta -:--:--
   --------------------------------------- 797.9/797.9 kB 17.3 MB/s eta 0:00:00
Using cached dnspython-2.7.0-py3-none-any.whl (313 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
from pymongo import MongoClient
from collections import defaultdict
import openpyxl
from urllib.parse import quote_plus

In [5]:
# Configura tu conexión

usuario = "jalope"
contrasena = "admin"
host = "127.0.0.1"
puerto = "27250"

uri = f"mongodb://{quote_plus(usuario)}:{quote_plus(contrasena)}@{host}:{puerto}/?directConnection=true"
client = MongoClient(uri)
db = client["tfm_db"]
n_docs = 1000

In [6]:
print(db.list_collection_names())

['INE_31304_POBLACION_RAW', 'pop_observed_raw', 'pop_projected_raw', 'pop_projections_nuts', 'pop_defunciones_raw', 'pop_nacimientos_raw']


In [7]:
# Inferir tipo de dato
def tipo_dato(v):
    if v is None:                   return "None"
    if isinstance(v, bool):         return "Boolean"
    if isinstance(v, int):          return "Integer"
    if isinstance(v, float):        return "Float"
    if isinstance(v, str):          return "String"
    if isinstance(v, list):         return "List"
    if isinstance(v, dict):         return "Object"
    if "ObjectId" in str(type(v)):  return "ObjectId"
    return type(v).__name__

In [8]:
# Recorremos colecciones y construir un DataFrames
resumen = {}
for col in db.list_collection_names():
    campos = defaultdict(list)
    for doc in db[col].find().limit(n_docs):
        for k, v in doc.items():
            campos[k].append(v)

    filas = []
    for k, vals in campos.items():
        ejemplos = list({str(x) for x in vals if x is not None})[:3]
        filas.append({
            "Campo":           k,
            "Tipo de Dato":    tipo_dato(vals[0]),
            "Requerido":       "",                     
            "Valores Ejemplo": ", ".join(ejemplos),
            "Descripción":     ""                      
        })
    resumen[col] = pd.DataFrame(filas)

In [11]:
# Exportar a un Excel con una hoja por colección
with pd.ExcelWriter("diccionario_datos_mongo.xlsx", engine="openpyxl") as writer:
    for nombre, df in resumen.items():
        sheet = nombre[:31]   # límite de 31 chars en nombres de hoja
        df.to_excel(writer, sheet_name=sheet, index=False)